# Automated setup of mixtures

We've been working on streamlining setup of simulations of arbitrary mixtures in AMBER/GROMACS/OpenMM and others for some of our own research. I thought I'd demo this really quick so you can get a feel for it and see if you're interested in contributing. It also allows quick setup and analysis of nontrivial liquid simulations, which can be a good opportunity to try out MDTraj and other analysis tools.

*Before running the below*, you will need to have followed the [getting started instructions](https://github.com/MobleyLab/drug-computing/blob/master/uci-pharmsci/getting-started.md) for this course.

In [2]:
from solvationtoolkit.solvated_mixtures import *

#In this particular instance I'll just look at six solutes/solvent mixtures (not an all-by-all combination) which are pre-specified
#solute names
solutes = ['phenol', 'toluene', 'benzene', 'methane', 'ethanol', 'naphthalene']
#Solvent names
solvents = ['cyclohexane', 'cyclohexane', 'cyclohexane', 'octanol', 'octanol', 'octanol']

#Number of solute/solvent molecules
Nsolu = 3
Nsolv = 100

#Construct systems
for idx in range( len( solutes) ):
    # Define new mixture
    mixture = MixtureSystem()
    # Add solute and solvent
    mixture.addComponent(name=solutes[idx], number=Nsolu)
    mixture.addComponent(name=solvents[idx], number=Nsolv)
    # Note you can optionally specify mole fraction instead, or a mix of numbers/mole fractions, etc.
    
    # Build system, including AMBER input files (but not GROMACS)
    mixture.build(amber=True, gromacs=True)


# Mixture

tolerance 2.000000
filetype pdb
output /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmphbi98883/tmpmumb08xh.pdb
add_amber_ter


structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmphbi98883/tmpm8xr8u7b.pdb
  number 3
  inside box 0. 0. 0. 24.788816 24.788816 24.788816
end structure

structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmphbi98883/tmpudv5wj6r.pdb
  number 100
  inside box 0. 0. 0. 24.788816 24.788816 24.788816
end structure


source leaprc.gaff
source oldff/leaprc.ff99SB
ZPW = loadmol2 in0.mol2
ZEQ = loadmol2 in1.mol2
box = loadPdb tbox.pdb
loadamberparams in0.frcmod
loadamberparams in1.frcmod
setbox box centers
saveAmberParm box out.prmtop out.inpcrd
quit


# Mixture

tolerance 2.000000
filetype pdb
output /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmp0b4h7bdz/tmpe6jdw1qf.pdb
add_amber_ter


structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmp0b4h7bdz/tmpaj_5foqf.pdb
  number 3
  inside box 0. 0. 0. 24.782739 24.782739 24.

In [53]:
from openff.evaluator.protocols.coordinates import BuildCoordinatesPackmol
from openff.evaluator.substances import Substance, Component, MoleFraction, ExactAmount
from openeye import oeiupac
from openeye import oechem
import shutil, os

#Number of solute/solvent molecules
Nsolu = 3
Nsolv = 100

#solute names
solutes = ['phenol', 'toluene', 'benzene', 'methane', 'ethanol', 'naphthalene']
#Solvent names
solvents = ['cyclohexane', 'cyclohexane', 'cyclohexane', 'octanol', 'octanol', 'octanol']

# Generate SMILES for solutes and solvents and store for use in building mixtures
solute_smiles = []
solvent_smiles = []
for name_pair in zip(solutes, solvents):
    #print(name_pair)
    solu_mol = oechem.OEMol()
    oeiupac.OEParseIUPACName(solu_mol, name_pair[0])
    solute_smiles.append(oechem.OECreateIsoSmiString(solu_mol))
    solv_mol = oechem.OEMol()
    oeiupac.OEParseIUPACName(solv_mol, name_pair[1])
    solvent_smiles.append(oechem.OECreateIsoSmiString(solv_mol))
    

# Storage for the mixtures we've built
mixtures = []
outdir = 'coordinate_files'
if not os.path.isdir(outdir): os.mkdir(outdir)
    
# Loop and build mixtures
for idx in range( len( solutes) ):
    # Define new mixture
    mixture_build = BuildCoordinatesPackmol("")
    substance = Substance()
    # Add solute and solvent
    substance.add_component(Component(solute_smiles[idx], role=Component.Role.Solute), ExactAmount(Nsolu))
    substance.add_component(Component(solvent_smiles[idx], role=Component.Role.Solvent), ExactAmount(Nsolv))
    #substance.add_component(Component("Cc1ccccc1", role=Component.Role.Solvent), MoleFraction(0.1))
    #substance.add_component(Component("C1CCCCC1", role=Component.Role.Solvent), MoleFraction(0.9))
    # Note you can optionally specify mole fraction instead, or a mix of numbers/mole fractions, etc.
    
    #substance.max_molecules = 150
    mixture_build.substance = substance

    #build
    mixture_build.execute()
    
    # Do file bookkeeping so the output files don't overwrite one another
    outfile = os.path.join(outdir, f'coordinate_file{idx}.pdb')
    shutil.copy( mixture_build.coordinate_file_path, outfile )
    mixture_build.coordinate_file_path = outfile
    
    #Store details
    mixtures.append(mixture_build)

    print(f"finished build {idx}")
    
    
print("Finished building")


finished build 0
finished build 1
finished build 2
finished build 3
finished build 4
finished build 5
Finished building


## Let's try and see if we can do a quick visualization of one of the systems via mdtraj just to make sure it looks right

In [54]:
#Import MDTraj
import mdtraj as md
#Load "trajectory" (structures)
#You can load from either format (SolvationToolkit generates both)
#traj = md.load( 'data/amber/phenol_cyclohexane_3_100.inpcrd', top = 'data/amber/phenol_cyclohexane_3_100.prmtop' )
traj = md.load( os.path.join(outdir, 'coordinate_file0.pdb'))#'data/gromacs/phenol_cyclohexane_3_100.gro')

#Input viewer
import nglview

#Set up view of structure
view = nglview.show_mdtraj(traj)

#Try some of the following to modify representations
view.clear_representations()
view.add_licorice('all')
# Select the first three "residues" of chain A for special display
# (Evaluator seems to make the first substance to be chain A, and then individual molecules are residues within that chain)
view.add_licorice('1:A or 2:A or 3:A', color = "blue") #NGLview has a whole selection lanuage
view.add_surface('1:A or 2:A or 3:A', opacity=0.3)

#Show the view. Note that this needs to be the last command used to manipulate the view, i.e. if you modify the
#representation after this, your view will be empty.
view

#VIEWER USAGE:
# - Use your typical zoom command/gesture (i.e. pinch) to zoom in and out 
# - Click and drag to reorient
# - Click on specific atoms/residues to find out details of what they are (and how they could be selected)

NGLWidget()

# Let's use a SMIRNOFF forcefield to parameterize the system, minimize, and run dynamics

(This requires `openforcefield`, which you will have conda-installed if you've followed the getting started info.)

First we handle imports

In [55]:
# Import the SMIRNOFF forcefield engine and some useful tools
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import openeye.oechem as oechem #Here we'll use OpenEye tookits, but RDKIt use is also possible

# We use PDBFile to get OpenMM topologies from PDB files
try:
    from openmm.app import PDBFile
except:
    from simtk.openmm.app import PDBFile

# MDTraj for working with trajectories; time for timing
import time
import mdtraj

## Now we handle assignment of force field parameters and generation of an OpenMM System

In [56]:
# Specify names of molecules that are components of the system
molnames = ['phenol', 'cyclohexane']

# Get molecules for components - required as input for OpenFF
oemols = []
for name in molnames:
    mol = oechem.OEGraphMol()
    oeiupac.OEParseIUPACName(mol, name)
    oemols.append(mol)

# Build set of OpenFF mols from OE molecules
OFFmols = []
for mol in oemols:
    OFFmols.append( Molecule.from_openeye(mol))
    
# Load OpenFF 2.0 force field
ff = ForceField('openff-2.0.0.offxml', 'test_forcefields/tip3p.offxml') 

# Get OpenMM topology for mixture of phenol and cyclohexane from where SolvationToolkit created
# it on disk (the first mixture we built)
pdbfile = PDBFile(mixtures[0].coordinate_file_path)

# Create OpenFF Topology
off_topology = Topology.from_openmm(openmm_topology = pdbfile.topology, unique_molecules = OFFmols)

# Assign SMIRNOFF parameters and create system
system = ff.create_openmm_system( off_topology)

## Finally we energy minimize and run dynamics

In [59]:
from datetime import datetime

# Set how many steps we'll run and other run parameters
num_steps=10000
trj_freq = 100 #Trajectory output frequency
data_freq = 100 #Energy/data output frequency
temperature = 300*unit.kelvin #Temperature
time_step = 2.*unit.femtoseconds
friction = 1./unit.picosecond #Langevin friction constant

# Bookkeeping -- if you run this more than once and perhaps encountered an exception, we need to make sure the reporter is closed
try: 
    reporter.close()
except: pass
    
# Set up integrator, platform for running simulation    
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)
platform = openmm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdbfile.topology, system, integrator)
# Set positions, velocities
simulation.context.setPositions(pdbfile.positions)
simulation.context.setVelocitiesToTemperature(temperature)

# Before doing dynamics, energy minimize (initial geometry will be strained)
simulation.minimizeEnergy()

# Set up reporter for output
reporter = mdtraj.reporters.HDF5Reporter('mixture.h5', trj_freq)
simulation.reporters=[]
simulation.reporters.append(reporter)
simulation.reporters.append(app.StateDataReporter('data.csv', data_freq, step=True, potentialEnergy=True, temperature=True, density=True))

# Run the dynamics
print("Starting simulation")
start = datetime.now()
simulation.step(num_steps)
end = datetime.now()

print("Elapsed time %.2f seconds" % (end-start).total_seconds())
#netcdf_reporter.close()
reporter.close()
print("Done!")


Starting simulation
Elapsed time 14.65 seconds
Done!


## Let's make a movie of our simulation

In [61]:
import nglview
traj=mdtraj.load('mixture.h5')
view = nglview.show_mdtraj(traj)

#Try some of the following to modify representations
view.clear_representations()
view.add_licorice('all')
view.add_licorice('1:A or 2:A or 3:A', color = "blue") 
view.add_surface('1:A', opacity=0.3)
view.add_surface('2:A or 3:A', color = 'red', opacity=0.3)

view #Note that if you view a movie and keep it playing, your notebook will run a hair slow...

NGLWidget(max_frame=99)

## Other possibly interesting things to try:
* Find the average distance from phenol to phenol
* Calculate the density or volume of the system
* etc.

(Drawing on MDTraj - see docs online)

In [3]:
# Use this box to try additional things